## Reading dome monitor with a labJack T7
Uses 2 analog input (AINs) to read the data.

Craig Lage - Jul 5, 2023

In [ ]:
import sys
import time, datetime
import numpy as np
import pandas as pd
import pickle as pkl
import matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta

from labjack import ljm  # Needed pip install labjack-ljm

In [ ]:
# Open LabJack T7
handle = ljm.openS("T7", "wifi", "139.229.170.164") # Shutter box
#handle = ljm.openS("T7", "wifi", "139.229.170.115") # Main box

In [ ]:
info = ljm.getHandleInfo(handle)
print("Opened a LabJack with Device type: %i, Connection type: %i,\n"
      "Serial number: %i, IP address: %s, Port: %i,\nMax bytes per MB: %i" %
      (info[0], info[1], info[2], ljm.numberToIP(info[3]), info[4], info[5]))

deviceType = info[0]
print(deviceType)

In [ ]:
# Setup and call eWriteNames to configure AIN0 and AIN1 on the LabJack.
# LabJack T7 and other devices configuration

# AIN0 and AIN1:
#   Negative channel = single ended (199)
#   Range: +/-10.0 V (10.0)
#   Resolution index = Default (0)
#   Settling, in microseconds = Auto (0)
names = ["AIN0_NEGATIVE_CH", "AIN0_RANGE", "AIN0_RESOLUTION_INDEX", "AIN0_SETTLING_US",
         "AIN1_NEGATIVE_CH", "AIN1_RANGE", "AIN1_RESOLUTION_INDEX", "AIN1_SETTLING_US"]
aValues = [199, 10.0, 0, 0,
           199, 10.0, 0, 0]
numFrames = len(names)
ljm.eWriteNames(handle, numFrames, names, aValues)

In [ ]:
print("\nSet configuration:")
for i in range(numFrames):
    print("    %s : %f" % (names[i], aValues[i]))


In [ ]:
# Set a long timeout to allow for web latency
timeout = 60.0
timeout_ms = int(timeout*1000.0)
ljm.writeLibraryConfigS('LJM_ETHERNET_SEND_RECEIVE_TIMEOUT_MS', timeout_ms)

In [ ]:
# Read AIN0 and AIN1 and AIN2
def readDomeMonitor():
    loopAmount = 1000
    # Read AIN's from the LabJack with eReadNames in a loop.
    names = ["AIN0"]
    numFrames = 1
    intervalHandle = 1
    ljm.startInterval(intervalHandle, 500000)  # Delay between readings (in microseconds)
    start = time.time()
    for n in range(loopAmount):
        results = ljm.eReadNames(handle, numFrames, names)
        ljm.waitForNextInterval(intervalHandle)
        stamp = time.time()
        delay = stamp - start
        start = stamp
        print(f"Time = {Time(stamp, format='unix_tai').utc.isot}, AIN0 = {results[0]:.3f}")
    return

In [ ]:
readDomeMonitor()

In [ ]:
def readStream(handle, names, readTime, scanRate=100):
    # This reads the accelerometers for a time readTime                                                             
    # and returns a Pandas timeSeries with the data                                                                 
    # names is the list of AIN ports (9 in total)                                                          
    # handle is the handle for talking to the labJack                                                               
    # scanRate is the read frequency in Hertz                                                                       
    # readTime is the total time of read in seconds                                                                 

    numAddresses = len(names)
    aScanList = ljm.namesToAddresses(numAddresses, names)[0]
    scansPerRead = int(scanRate * readTime)
    # Configure and start stream                                                                                    
    scanRate = ljm.eStreamStart(handle, scansPerRead, numAddresses, aScanList, scanRate)
    start = datetime.datetime.now()
    # Stream the data                                                                                               
    ret = ljm.eStreamRead(handle)
    # Stop the stream                                                                                               
    ljm.eStreamStop(handle)
    aData = ret[0]
    # Reshape the data                                                                                              
    newData = np.resize(aData, (scansPerRead, numAddresses))
    end = start + datetime.timedelta(seconds = readTime)
    date_rng = pd.date_range(start=start, end=end, periods=scansPerRead)
    # Create the Pandas dataFrame                                                                                   
    df = pd.DataFrame(newData, index=date_rng,
                      columns=['AIN0', 'AIN1'])
    # Pickle the dataframe.  Use a temporary filename,                                                              
    # then update it after the exposure has finished.   
    # Stuff to rename file                                                                                                                 
    name = start.strftime("%Y-%m-%dT%H:%M:%S")
    name = name.replace('-','').replace(':','')
    name = name + 'Z'
    filename = f'/scratch/labJackData/Dome_{name}.pkl'
    file = open(filename, 'wb')
    pkl.dump(df, file)
    file.close()

    finish = datetime.datetime.now()
    print(f"Finishing Accel data: {finish}")
    return


In [ ]:
names = ["AIN0", "AIN1"]
readStream(handle, names, 30.0, scanRate=100)

In [ ]:
file = open(f'/scratch/labJackData/Dome_20230706T005245Z.pkl', 'rb')
df = pkl.load(file)

In [ ]:
df.head(5)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,10))
df['AIN0'].plot(ax=ax, label="Close signal")
plt.ylim(-10,10)
plt.legend()

In [ ]:
names = ["AIN0", "AIN1"]
for i in range(20):
    readStream(handle, names, 1.0, scanRate=100)


In [ ]:
ljm.eStreamStop(handle)

In [ ]:
try:
    ljm.eStreamStop(handle)
except:
    pass

In [ ]:
test = ljm.close(handle)
print(test)

In [ ]:
ljm.closeAll()

In [ ]:
finish_time = Time("2023-08-17T18:00", format='isot')

In [ ]:
time.time() - finish_time.unix_tai

In [ ]:
ljm.LJMError

In [ ]:
ljm.closeAll()

In [ ]:
ljm.openS?

In [ ]:
ljm.

In [ ]:
ljm.log()

In [ ]:
for i in range(5):
    print(i)
    time.sleep(2.0)